# Exam - Rest Server

In [1]:
import cherrypy
import json
import redis
import uuid
from redis.commands.json.path import Path
from datetime import datetime

REDIS_HOST = "redis-17243.c250.eu-central-1-1.ec2.cloud.redislabs.com"
REDIS_PORT = 17243
REDIS_USER = "default"
REDIS_PSSW = "IIRJyR2QEWYULfX8g9ZTXAAeG3shxYsm"
redis_client = redis.Redis(REDIS_HOST, REDIS_PORT, password=REDIS_PSSW, username=REDIS_USER,)

In [2]:
#online/
class Status(object):
    exposed = True
    #GET
    def GET(self, *path, **query): #* is a list and ** a dictionary
        response_dict = {
            'status': 'online'
        }

        response = json.dumps(response_dict)

        return response
#devices/
class DevicesList(object):
    exposed = True
    def GET(self, *path, **query): #* is a list and ** a dictionary
            
        #we retrive parameters from the query dictionary
        #use NONE to avoid error because the parameters are optional!

        message = query.get('message', None)
        #batter and power are always sent together, so we can check only for battery
        keys = redis_client.keys(f'0x*:prediction')
        items = list(set([item.decode().split(':')[0] for item in keys]))

        res_json = {'mac_addresses': items}
        response = json.dumps(res_json)

        return response
        

In [3]:
#device/{mac_address}
class Device(object):
    exposed = True
    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, message = 'Bad request: missing MAC address')
            
        from_ts = query.get('from', None)
        if from_ts is None:
            raise cherrypy.HTTPError(400, message = 'Bad request: missing start time')
        to_ts = query.get('to', None)
        if to_ts is None:
            raise cherrypy.HTTPError(400, message = 'Bad request: missing end time')
        mac_address = path[0]
        
        found = redis_client.keys(f'{mac_address}:prediction')
        if not found:
            raise cherrypy.HTTPError(404, message = 'Not found: invalid MAC address')

        timestamps = [t[0] for t in redis_client.ts().range(f'{mac_address}:prediction', from_ts, to_ts)]
        predictions = [t[1] for t in redis_client.ts().range(f'{mac_address}:prediction', from_ts, to_ts)]
        probabilities = [t[1] for t in redis_client.ts().range(f'{mac_address}:probability', from_ts, to_ts)]
        latencies = [t[1] for t in redis_client.ts().range(f'{mac_address}:latency', from_ts, to_ts)]
        
        res = {
            'mac_address': mac_address,
            'timestamps': timestamps,
            'predictions': predictions,
            'probabilities': probabilities,
            'latencies': latencies
        }

        response = json.dumps(res)
        return response

    def DELETE(self, *path, **query):
        '''     
            200 – OK: Everything worked as expected. 
            400 – Bad Request: missing MAC address. 
            404 – Not Found: invalid MAC address.
        '''    
            
        if not path:
            raise cherrypy.HTTPError(400, message = 'Bad request: missing MAC address')
        else:
            mac_address = path[0]
            keys = redis_client.keys(f'{mac_address}:*')
            deleted = redis_client.delete(*keys)

            if not keys or not deleted:
                raise cherrypy.HTTPError(404, message = 'Not Found: invalid MAC address')
        return

In [4]:
#spy/{mac_address}
class Spy(object):
    exposed = True
    def GET(self, *path, **query):
        if len(path) != 1:
            raise cherrypy.HTTPError(400, message = 'Bad request: missing MAC address')

        mac_address = path[0]
        
        found = redis_client.keys(f'{mac_address}:prediction')
        if not found:
            raise cherrypy.HTTPError(404, message = 'Not found: invalid MAC address')

        last = redis_client.ts().get(f'{mac_address}:prediction')
        date = datetime.fromtimestamp(last[0]/1000)
        prediction = last[1]
        LABELS = [
            "Bus",
            "Minibus",
            "Pickup",
            "Sports Car",
            "Jeep",
            "Truck",
            "Crossover",
            "Car (C Class 4K)"
        ]

        res = {
            'date': date.strftime('%a %d %b %Y, %I:%M%p'),
            'prediction': LABELS[int(prediction)]
        }

        response = json.dumps(res)
        return response

In [5]:
if __name__ == '__main__':
    #This is done to redirect the routes.
    conf = {
        '/' : {'request.dispatch' : cherrypy.dispatch.MethodDispatcher()}
    }
    #mounting the API route
    cherrypy.tree.mount(Status(), '/online', conf)
    cherrypy.tree.mount(DevicesList(), '/devices', conf)
    cherrypy.tree.mount(Device(), '/device', conf)
    cherrypy.tree.mount(Spy(), '/spy', conf)
    cherrypy.config.update({'server.socket_host' : '0.0.0.0'})
    cherrypy.config.update({'server.socket_port' : 8080}) #to make it work on deepnote, use 8080

    cherrypy.engine.start()
    cherrypy.engine.block()

[12/Jun/2023:15:52:27] ENGINE Bus STARTING
[12/Jun/2023:15:52:27] ENGINE Started monitor thread 'Autoreloader'.
[12/Jun/2023:15:52:27] ENGINE Serving on http://0.0.0.0:8080
[12/Jun/2023:15:52:27] ENGINE Bus STARTED
172.3.22.214 - - [12/Jun/2023:15:52:32] "GET /spy/0x2c8db1c3edca HTTP/1.1" 200 64 "" "python-requests/2.28.1"
[12/Jun/2023:16:14:15] ENGINE Keyboard Interrupt: shutting down bus
[12/Jun/2023:16:14:15] ENGINE Bus STOPPING
[12/Jun/2023:16:14:16] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('0.0.0.0', 8080)) shut down
[12/Jun/2023:16:14:16] ENGINE Stopped thread 'Autoreloader'.
[12/Jun/2023:16:14:16] ENGINE Bus STOPPED
[12/Jun/2023:16:14:16] ENGINE Bus EXITING
[12/Jun/2023:16:14:16] ENGINE Bus EXITED
[12/Jun/2023:16:14:16] ENGINE Waiting for child threads to terminate...


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=05dea53b-e624-4a45-a48c-4f8a58ba823b' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>